In [2]:
import ctypes
import numpy as np
import math
import time

#Define point structure
class HeliosPoint(ctypes.Structure):
    #_pack_=1
    _fields_ = [('x', ctypes.c_uint16),
                ('y', ctypes.c_uint16),
                ('r', ctypes.c_uint8),
                ('g', ctypes.c_uint8),
                ('b', ctypes.c_uint8),
                ('i', ctypes.c_uint8)]
class Dac:
    def __init__(self):
        #Load and initialize library
        self.HeliosLib = ctypes.cdll.LoadLibrary("./libHeliosDacAPI.so")
        self.num_devices = self.HeliosLib.OpenDevices()
        print("Found ", self.num_devices, "Helios DACs")
        # Define limits
        self.xy_max = int(2**12-1)

class DacQueue:
    '''
    A queue for patterns sent to the dac. 
    Performs smart stitching between patterns.
    '''
    def __init__(self):
        # Dac object for this queue
        self.dac = Dac()
        # The last position of the last pattern (x,y)
        self.last_pos = (0,0)
        # Sample rate of the DAC
        self.dac_rate = 55000
        # Debugging dac rate
        #self.debug_rate = 550
        self.debug_rate = 150
        
    
    def submit(self, pat_pos, pat_col, angular_density=100, debug=False):
        '''
        Submits a new pattern to the dac, transitioning smoothly from the last one.
        Angular density describes how many points per radian should be used to transition
        '''
        # Make the transition
        dist = np.sqrt(np.sum(np.power(self.last_pos-pat_pos[0,:], 2)))
        num_gap_points = int(dist*angular_density)

        gap_pos = np.linspace(self.last_pos, pat_pos[0,:], num_gap_points, dtype=np.float)
        gap_col = np.zeros((num_gap_points, 3), dtype=np.float)
        if(debug):
            gap_col = np.ones_like(gap_col)/4

        # Prep the transition pattern
        gap_points, num_gap_points = self.prep_pattern(gap_pos, gap_col, gap=True)
        
        # Prep the new pattern
        pat_points, num_pat_points = self.prep_pattern(pat_pos, pat_col)
        
        # Send the transition pattern to the dac
        self.write_frames(gap_points, num_gap_points, do_not_loop=True, debug=debug)
        # Send the new pattern to the dac
        self.write_frames(pat_points, num_pat_points, do_not_loop=True, debug=debug)
        # Set the last_position
        self.last_pos = pat_pos[-1, :].copy()
        
    def prep_pattern(self, arr_pos, arr_col, gap=False):
        '''
        1) Scales and color corrects 
        2) Converts from unit space to DAC coordinates
        3) Produces a DAC compatible frame and displays it
        '''
        # Force pass-by-value
        arr_pos = arr_pos.copy()
        arr_col = arr_col.copy()
        # Scale the pattern 
        arr_pos = position_corection(arr_pos)
        # Format the position array (-1.0-1.0) -> int(0-4095)
        arr_pos = (((arr_pos+1)/2.0)*self.dac.xy_max).astype(np.int32)

        # Perform color correction if not gap points
        if(not gap):
            arr_col = color_correction(arr_col)
        # Format the color array (0-1.0) -> int(0-255)
        arr_col = (arr_col*255).astype(np.int32)
        # Fill a heliospoint arr with these values
        num_points = len(arr_pos)

        frameType = HeliosPoint * num_points
        points = frameType()
        # Fill the frame
        for idx in range(num_points):
            points[idx] =     HeliosPoint(int(arr_pos[idx, 0]), 
                                         int(arr_pos[idx, 1]), 
                                         int(arr_col[idx, 0]), 
                                         int(arr_col[idx, 1]),
                                         int(arr_col[idx, 2]),
                                         int(255))
        
        return points, num_points
            
    def write_frames(self, points, num_points, do_not_loop=False, start_immediately=False, debug=False):
        if(num_points < 1):
            return
        # Write the values out 
        status_attempts = 0
        max_attempts = np.inf
        # Make 512 attempts for DAC status to be ready. After that, just give up and try to write the frame anyway
        while(status_attempts < max_attempts and self.dac.HeliosLib.GetStatus(0) != 1):
            status_attempts += 1
            
        # Create flags 
        flags = do_not_loop << 1 | start_immediately << 0
        # Send to DAC
        frame_rate = self.debug_rate if debug else self.dac_rate
        self.dac.HeliosLib.WriteFrame(0, frame_rate, flags, ctypes.pointer(points), num_points)

def color_correction(arr_color):
    '''
    Corrects the nonlinearities in the color curve 
    '''
    non_zero = arr_color > 0
    # Scale to the lower cutoff (R, G, B)
    lower_cutoff = np.array([0.25, 0.09, 0.09])
    arr_color = arr_color*(1-lower_cutoff) + lower_cutoff
    
    # Cutoff any zeros to ensure real black
    arr_color *= non_zero
    
    return arr_color

def position_corection(arr_pos):
    '''
    Corrects x-y directions so that (x=-1,y=-1) is bottom left
    Scales the entire pattern so that it does not clip (amps are weird)
    '''
    # Reverse direction of x and y
    arr_pos *= -1
    
    max_scale = 0.75
    return arr_pos*max_scale
    
def make_circular(arr_pos, arr_color):
    return np.concatenate([arr_pos, arr_pos[::-1, :]]), np.concatenate([arr_color, arr_color[::-1, :]])

def connect_in_space(arr_pos_list, angular_density=100, wait_per=None):
    '''
    Given a list of position arrays, produce a list of array positions that are connected
    '''
    new_arr_pos_list = []
    for i in range(len(arr_pos_list)-1):
        # Get start/end positions for this gap
        pos_start = arr_pos_list[i][-1, :].copy()
        pos_end = arr_pos_list[i+1][ 0, :].copy()
        
        # Make the transition
        dist = np.sqrt(np.sum(np.power(pos_start-pos_end, 2)))
        num_gap_points = int(dist*angular_density)
        gap_pos = np.linspace(pos_start, pos_end, num_gap_points, dtype=np.float)
        
        # If we have a wait period, add it to the end of the gap positions
        if(wait_per):
            rep_bef = np.repeat(pos_start[np.newaxis, :], wait_per, axis=0)
            rep_aft = np.repeat(pos_end[np.newaxis, :], wait_per, axis=0)
            gap_pos = np.concatenate([rep_bef, gap_pos, rep_aft])
            
        
        new_arr_pos_list.append(arr_pos_list[i])
        new_arr_pos_list.append(gap_pos.copy())
    new_arr_pos_list.append(arr_pos_list[-1])
    
    return new_arr_pos_list

def fixed_interp(points, n_interp_points):
    '''
    Expands a list of points to a position array where those 
    points are the anchors in a smooth trajectory. 
    There is a fixed number of interp points between 
    the given points. 
    Inputs: 
        points - (N,2) - list of anchor points
        n_interp_points - number of points between anchors
    '''
    interped_pos_arr = []
    for i in range(len(points)-1):
        p1 = points[i]
        p2 = points[i+1]
        gap_pos = np.linspace(p1, p2, n_interp_points)
        interped_pos_arr += [p1]
        interped_pos_arr += [gap_pos]
    interped_pos_arr += [p2]
    
    # Convert them into one array 
    interped_pos_arr = np.concatenate(interped_pos_arr, axis=0)
    return interped_pos_arr
    


In [3]:
queue = DacQueue()

Found  1 Helios DACs


In [3]:
# Produce slow circles 
T = 1000
freq = 1.0/100.0
color_arr = np.ones((T,3))/10
start_ang = 0

while(True):
    theta = np.linspace(start_ang, start_ang+2*np.pi, T)
    pos_arr = np.zeros((T, 2))
    pos_arr[:, 0] = np.cos(freq*theta)
    pos_arr[:, 1] = np.sin(freq*theta)
    queue.submit(pos_arr, color_arr)
    start_ang += 2*np.pi

KeyboardInterrupt: 

In [5]:
# Produce colored circles 
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)

start = (0.1, 1.0, 0.1)
finish = (1.0, 0.1, 1.0)
smooth = np.linspace(start, finish, T//2)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr)


In [9]:
# Produce a SINE wave
T = 1000
theta = np.linspace(0, 2*np.pi, T)
freq = 3
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.linspace(-1.0, 1.0, T)
pos_arr[:, 1] = np.sin(theta*freq)

start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr, debug=False)


In [8]:
# Produce a phase shifting sine wave
T = 600
freq = 4
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.linspace(-1.0, 1.0, T)
start = (0.0, 0.0, 1.0)
finish = (1.0, 1.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(10):
    for phi in np.linspace(0, 2*np.pi, 100):
        theta = np.linspace(phi, phi+2*np.pi, T)

        pos_arr[:, 1] = np.sin(theta*freq)

        queue.submit(pos_arr, color_arr, debug=False)


In [10]:
# Produce a double SINE wave
T = 1000
theta = np.linspace(0, 4*np.pi, T)
freq = 3
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.concatenate([np.linspace(-1.0, 1.0, T//2), np.linspace(1.0, -1.0, T//2)])
pos_arr[:, 1] = np.sin(theta*freq)

start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr, debug=False)


In [14]:
# Produce a modulated wave
T = 1000
theta = np.linspace(0, 4*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.concatenate([np.linspace(-1.0, 1.0, T//2), np.linspace(1.0, -1.0, T//2)])
start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4
for i in range(3):
    for freq in np.linspace(1, 4, 200):
        pos_arr[:, 1] = np.sin(theta*freq)
        queue.submit(pos_arr, color_arr, debug=False)


In [147]:
# Make a white fan with colored beams in it

beam_dwell = 50
n_beams = 6
beam_col = (0.0, 1.0, 0.0)
fan_col = np.array((1.0, 1.0, 1.0))/4

beam_positions = np.linspace(-1, 1, n_beams)
beams = []
for pos in beam_positions:
    beam = np.zeros((beam_dwell, 2))
    beam[:, 0] = pos
    beams.append(beam.copy())

beam_fan_list = connect_in_space(beams, wait_per=10)
color_arrs = []
for idx,pat in enumerate(beam_fan_list):
    if(idx % 2 == 0):
        color_arrs.append(np.ones((pat.shape[0], 3))*beam_col)
    else:
        color_arrs.append(np.ones((pat.shape[0], 3))*fan_col)
        
pos_arr = np.concatenate(beam_fan_list, axis=0)
col_arr = np.concatenate(color_arrs, axis=0)

pos_arr, col_arr = make_circular(pos_arr, col_arr)

offset_arr = np.zeros_like(pos_arr) 
for i in range(4):
    for y_pos in np.concatenate([np.linspace(-1, 1, 30), np.linspace(1, -1, 60)]):
        offset_arr[:, 1] = y_pos
        queue.submit(pos_arr+offset_arr, col_arr, debug=False)

KeyboardInterrupt: 

In [148]:
# Make a white cone with colored beams in it

beam_dwell = 50
n_beams = 6
beam_col = (0.0, 1.0, 0.0)
fan_col = np.array((1.0, 1.0, 1.0))/4

thetas = np.linspace(0, 2*np.pi, n_beams)
beam_positions = np.stack([np.cos(thetas), np.sin(thetas)], axis=1)
beams = []
for pos in beam_positions:
    beam = np.zeros((beam_dwell, 2))
    beam[:, 0:2] = pos
    beams.append(beam.copy())

beam_fan_list = connect_in_space(beams, wait_per=10)
color_arrs = []
for idx,pat in enumerate(beam_fan_list):
    if(idx % 2 == 0):
        color_arrs.append(np.ones((pat.shape[0], 3))*beam_col)
    else:
        color_arrs.append(np.ones((pat.shape[0], 3))*fan_col)
        
pos_arr = np.concatenate(beam_fan_list, axis=0)
col_arr = np.concatenate(color_arrs, axis=0)

for i in range(100):
    for y_pos in np.concatenate([np.linspace(-1, 1, 2), np.linspace(1, -1, 2)]):
        queue.submit(pos_arr, col_arr, debug=False)

In [152]:
# Make a white cone with colored beams in it that randomly mutates
beam_dwell = 50
n_beams = 3
beam_col = (0.0, 1.0, 0.0)
fan_col = np.array((1.0, 1.0, 1.0))/8

def normalize_color(color):
    d = math.sqrt(np.sum(color**2))
    return color/d

T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)

thetas = np.linspace(0, 2*np.pi, n_beams+1)
beam_positions = np.stack([np.cos(thetas), np.sin(thetas)], axis=1)
beams = []
for pos in beam_positions:
    beam = np.zeros((beam_dwell, 2))
    beam[:, 0:2] = pos
    beams.append(beam.copy())
beam_fan_list = connect_in_space(beams, wait_per=10)

pattern_size = 0.8
steps = 10000
ang = 0
for step in range(steps):
    ang += 0.01*np.pi
    rot_mat = np.array([[np.cos(ang), -np.sin(ang)], 
                        [np.sin(ang), np.cos(ang)]])
    
    rnd = 0.1*(np.random.rand(3)-0.5)
    beam_col += rnd
    beam_col = np.clip(beam_col, 0, 1)
    beam_col = normalize_color(beam_col)
    color_arrs = []
    for idx,pat in enumerate(beam_fan_list):
        if(idx % 2 == 0):
            color_arrs.append(np.ones((pat.shape[0], 3))*beam_col)
        else:
            color_arrs.append(np.ones((pat.shape[0], 3))*fan_col)

    pattern_size = np.clip(pattern_size + ((np.random.rand(1) > 0.5).astype(np.float32)-0.5)/20, 0.2, 1.0) 
    pos_arr = np.concatenate(beam_fan_list, axis=0)
    col_arr = np.concatenate(color_arrs, axis=0)
    queue.submit(pattern_size*np.matmul(pos_arr, rot_mat), col_arr, debug=False)

KeyboardInterrupt: 

In [1]:
# Circle w/ a spike
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)
start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T//2)
color_arr = np.concatenate([smooth, smooth[::-1]])/4

spike_freq = 20
for i in range(T):
    pos_copy = pos_arr.copy()
    p = (spike_freq*i)%T
    pos_copy[int(p), :] = 0
    queue.submit(pos_copy, color_arr, debug=False)

NameError: name 'np' is not defined

In [ ]:
np.matmul(np.zeros((T,2)), np.zeros((2,2))).shape

In [ ]:
# Make a smart circle that maps color and radius to some input

# Dynamic range of r
min_r = 0.8
max_r = 1.0
# Correspond to the freq components to visualize
bins = 16
T = 1000
theta = np.linspace(np.pi/2, np.pi/2+2*np.pi, 2*T)

col_arr = np.array((0.5, 0.2, 1.0))[np.newaxis, :]

# Perform the loop 
for i in range(1000):
    # Since we don't have sound input, make it up 
    if(i%200 == 0):
        freq_hist = np.random.rand(bins)*(max_r-min_r) + min_r
        interp_len = int((T-bins)/(bins+1) + 1)
        #r = fixed_interp(freq_hist, n_interp_points=interp_len)[0:T]
        # The array holding the radius of the circle at an angle
        r = np.repeat(freq_hist, T//bins+1)[0:T]
        # Make symmetric
        r = np.concatenate([r, r[::-1]])
    

    
    pos_arr = np.zeros((2*T, 2))
    pos_arr[:, 0] = r*np.cos(theta)
    pos_arr[:, 1] = r*np.sin(theta)
    queue.submit(pos_arr, col_arr*np.ones((2*T, 3)), debug=False)




In [ ]:
freq_hist = np.random.rand(bins)*(max_r-min_r) + min_r
# Find the positions of the components in the circle
r_positions = np.linspace(np.pi/2, np.pi/2+2*np.pi, bins+1, endpoint=False) + 1/(2*(bins+1))
pos = np.stack([np.sin(r_positions), np.sin(r_positions)], axis=1)
fixed_interp(pos, 10)

In [ ]:
np.repeat(freq_hist, 100)[0:T].shape


In [ ]:
# Make a tri-colored circle
# Make a tricolor circle
T = 1000
pos_arr = np.zeros((T, 2))
col_arr = np.zeros((T, 3))

# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
pos_arr[:,0] = np.cos(theta)
pos_arr[:,1] = np.sin(theta)


# Tri color the circle
col_arr[theta < np.pi, 0] = 0
col_arr[theta < np.pi, 1] = 0
col_arr[theta < np.pi, 2] = 0.3
col_arr[theta < (2/3)*np.pi, 0] = 0
col_arr[theta < (2/3)*np.pi, 1] = 0.3
col_arr[theta < (2/3)*np.pi, 2] = 0
col_arr[theta < (1/3)*np.pi, 0] = 0.3
col_arr[theta < (1/3)*np.pi, 1] = 0
col_arr[theta < (1/3)*np.pi, 2] = 0

for i in range(100):
    queue.submit(pos_arr, color_arr)


In [ ]:
scales = np.concatenate([np.linspace(1.0, 0.5, 10), np.linspace(0.5, 1, 20)])

import time
while(True):
    for scale in scales:
        new_pos_arr = scale*pos_arr
        queue.submit(new_pos_arr, col_arr)
    break


In [43]:
import cv2
def draw_bitmap(arr_pos, arr_color, dims=(400,400)):
    '''
    Draws a pattern as if we were drawing it on a wall
    '''
    # Copy the arrays so we don't change them 
    arr_pos = arr_pos.copy()
    arr_color = arr_color.copy()
    # Map the positions
    arr_pos[:,0] *= -1
    arr_pos = (((arr_pos+1)/2)*dims).astype(np.int)
    # Map the colors
    arr_color = arr_color[:,::-1]
    T = len(arr_pos)
    img = np.zeros(dims+(3,))
    # Draw points
    for t in range(1, T):
        cv2.line(img, arr_pos[t-1], arr_pos[t], color=arr_color[t-1], thickness=2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
    

In [ ]:
# Make three parallel lines
T_line = 100

arr_pos_list = []
arr_color_list = []
for c_idx, y in enumerate(range(-1, 2, 1)):
    arr_pos_list.append(np.stack([np.linspace(-1, 1, T_line), y*np.ones(T_line)], axis=1))
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())

for i in range(100):
    for arr_pos, arr_color in zip(arr_pos_list, arr_color_list):
        queue.submit(arr_pos, arr_color)

In [ ]:
# Make three concentric circles
T_line = 200
theta_line = np.linspace(0, 2*np.pi, T_line)

arr_pos_list = []
arr_color_list = []
for c_idx, scale in enumerate([0.9, 0.95, 1]):
    arr_pos_list.append(np.stack([np.cos(theta_line), np.sin(theta_line)], axis=1)/scale)
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())

for i in range(1000):
    for arr_pos, arr_color in zip(arr_pos_list, arr_color_list):
        queue.submit(arr_pos, arr_color)

In [ ]:
draw_bitmap(arr_pos, arr_color)

In [31]:
# make a sine wave
T = 500
freq = 4
theta_line = np.linspace(0, 2*np.pi, T)
arr_pos = np.stack([np.linspace(-1, 1, T), np.sin(freq*theta_line)], axis=1)
#arr_color = np.ones((T, 3))
arr_color = np.linspace((0.1, 1.0, 0.3), (1.0, 0.0, 0.7), T)
for i in range(100):
    queue.submit(arr_pos, arr_color, debug=False)

In [32]:
from skimage.color import rgb2lab, lab2rgb

In [95]:
print(lab2rgb((0,0,0)))
print(rgb2lab((0.10,0.10,0.255)))
print(rgb2lab((0.5,1,1)))
print('this: ', lab2rgb((10.0,-100.0, 10.0)))



l_lims = (0, 100)
a_lims = (-100, 100)
b_lims = (-100, 100)

w = a_lims[1] - a_lims[0] 
h = b_lims[1] - b_lims[0] 
img = np.zeros((h, w, 3), np.float32)

for a_idx, a in enumerate(np.linspace(*a_lims, h)):
    for b_idx, b in enumerate(np.linspace(*b_lims, w)):
        img[a_idx, b_idx, :] = lab2rgb((90.0, float(a), float(b)))

cv2.imshow('frame', img)
cv2.waitKey(0)

[0. 0. 0.]
[ 11.357263    13.5097441  -25.23433865]
[ 93.13904229 -35.33482179 -10.88977706]
this:  [0.         0.20535174 0.02948849]


106

In [97]:
lab2rgb(np.random.rand(100, 100, 3))

lab_img = 
cv2.imshow('f', lab2rgb(np.linspace((50, -127, 127))))

array([[[2.57922176e-02, 1.12165309e-02, 5.05683250e-03],
        [4.37358992e-03, 4.53428421e-04, 0.00000000e+00],
        [1.28798169e-02, 6.98422447e-03, 6.68063320e-03],
        ...,
        [1.74350785e-02, 1.08569837e-02, 6.70971081e-03],
        [1.33796085e-02, 0.00000000e+00, 0.00000000e+00],
        [1.33085430e-02, 6.00047213e-04, 0.00000000e+00]],

       [[2.48174935e-02, 7.14020560e-03, 2.06070348e-03],
        [1.46171200e-02, 8.27866722e-03, 3.34512197e-03],
        [1.38298105e-02, 0.00000000e+00, 0.00000000e+00],
        ...,
        [6.22313450e-03, 0.00000000e+00, 0.00000000e+00],
        [5.60586841e-03, 1.26265281e-03, 0.00000000e+00],
        [1.86176357e-02, 1.23693192e-02, 1.19535747e-02]],

       [[1.70000118e-02, 4.59894197e-03, 0.00000000e+00],
        [1.31644877e-02, 9.42564665e-03, 9.99510596e-03],
        [2.06500717e-02, 7.96139309e-03, 6.62717268e-03],
        ...,
        [1.20577795e-02, 8.82497700e-04, 0.00000000e+00],
        [2.50245896e-02, 9.28

In [ ]:
# Circles in the CIELAB Space
# UNFINISHED - see this for documentation: https://github.com/scikit-image/scikit-image/blob/main/skimage/color/colorconv.py#L1142-L1183
# Range is here? :https://stackoverflow.com/questions/25294141/cielab-color-range-for-scikit-image
scales = np.concatenate([np.linspace(1, 0.1, 200), np.linspace(0.1, 1, 200)])

T = 400


# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
arr_pos = np.zeros((T, 2))
arr_pos[:,0] = np.cos(theta)
arr_pos[:,1] = np.sin(theta)

# Set the colors 
arr_cols = []
for scale in scales:
    arr_col_lab = np.zeros((T, 3))
    # Set the luminosity to the scale
    #arr_col_lab[:, 0] = scale*100.0
    arr_col_lab[:, 0] = 10
    arr_col_lab[:, 1::] = 127.0*arr_pos/2.0
    arr_cols.append(lab2rgb(arr_col_lab))

for i in range(1000):
    for scale, arr_col in zip(scales, arr_cols):
        queue.submit(arr_pos*scale, arr_col)

    
    

In [110]:
img = np.zeros((10000, 10000, 2))
for row in range(10000):
    for col in range(10000):
        img[row, col] = row, col

KeyboardInterrupt: 

In [108]:
img.shape

(100, 100, 2)

In [128]:
N = 10
x = np.linspace(0, N-1, N)
y = np.linspace(0, N-1, N)
x = np.stack([x]*N, axis=1)
y = np.stack([y]*N, axis=0)
stacked = np.stack([x, y], axis=-1)

In [129]:
print(stacked)

[[[0. 0.]
  [0. 1.]
  [0. 2.]
  [0. 3.]
  [0. 4.]
  [0. 5.]
  [0. 6.]
  [0. 7.]
  [0. 8.]
  [0. 9.]]

 [[1. 0.]
  [1. 1.]
  [1. 2.]
  [1. 3.]
  [1. 4.]
  [1. 5.]
  [1. 6.]
  [1. 7.]
  [1. 8.]
  [1. 9.]]

 [[2. 0.]
  [2. 1.]
  [2. 2.]
  [2. 3.]
  [2. 4.]
  [2. 5.]
  [2. 6.]
  [2. 7.]
  [2. 8.]
  [2. 9.]]

 [[3. 0.]
  [3. 1.]
  [3. 2.]
  [3. 3.]
  [3. 4.]
  [3. 5.]
  [3. 6.]
  [3. 7.]
  [3. 8.]
  [3. 9.]]

 [[4. 0.]
  [4. 1.]
  [4. 2.]
  [4. 3.]
  [4. 4.]
  [4. 5.]
  [4. 6.]
  [4. 7.]
  [4. 8.]
  [4. 9.]]

 [[5. 0.]
  [5. 1.]
  [5. 2.]
  [5. 3.]
  [5. 4.]
  [5. 5.]
  [5. 6.]
  [5. 7.]
  [5. 8.]
  [5. 9.]]

 [[6. 0.]
  [6. 1.]
  [6. 2.]
  [6. 3.]
  [6. 4.]
  [6. 5.]
  [6. 6.]
  [6. 7.]
  [6. 8.]
  [6. 9.]]

 [[7. 0.]
  [7. 1.]
  [7. 2.]
  [7. 3.]
  [7. 4.]
  [7. 5.]
  [7. 6.]
  [7. 7.]
  [7. 8.]
  [7. 9.]]

 [[8. 0.]
  [8. 1.]
  [8. 2.]
  [8. 3.]
  [8. 4.]
  [8. 5.]
  [8. 6.]
  [8. 7.]
  [8. 8.]
  [8. 9.]]

 [[9. 0.]
  [9. 1.]
  [9. 2.]
  [9. 3.]
  [9. 4.]
  [9. 5.]
  [9. 6.]
  [9. 7.]
  

In [132]:
stacked[9, 1]

array([9., 1.])

In [16]:
# Random ellipses
T = 500
arr_col = np.ones((T, 3))
while(True):
    a = np.random.rand(1)
    b = np.random.rand(1)
    theta = np.linspace(0, 2*np.pi, T)
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = a*np.cos(theta)
    arr_pos[:,1] = b*np.sin(theta)
    col_gate = np.repeat(np.random.rand(T//10, 3) > 0.2, 10, axis=0)
    for i in range(20):
        queue.submit(arr_pos, arr_col*col_gate)

    

KeyboardInterrupt: 

In [79]:
# Interpolated ellipses
T = 500
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

a_old = 1.0
b_old = 1.0
c_old = (1.0, 1.0, 1.0)
arr_pos = np.zeros((T, 2))
arr_pos[:,0] = np.cos(theta)
arr_pos[:,1] = np.sin(theta)
accel_max = math.sqrt((1-0)**2 + (1-0)**2)

T_trans = 5

while(True):
    a = np.random.rand(1)[0]
    b = np.random.rand(1)[0]
    c = np.random.rand(3)
    
    #col_gate = np.repeat(np.random.rand(T//10, 3) > 0.5, 10, axis=0)
    col_gate = 1
    ell_interp = np.linspace((a_old, b_old), (a, b), T_trans)
    col_interp = np.linspace(c_old, c, T_trans, axis=0)
    accel = math.sqrt((a_old-a)**2 + (b_old-b)**2)
    accel_norm = accel/accel_max

    for ell_inter, c_inter in zip(ell_interp, col_interp):
        queue.submit((ell_inter*arr_pos), np.repeat(c_inter[np.newaxis, :], T, axis=0))
        
    a_old = a
    b_old = b
    c_old = c

    

KeyboardInterrupt: 

In [84]:
# Interpolated ellipses with beamifier (dweller)
T = 500
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

a_old = 1.0
b_old = 1.0
c_old = (1.0, 1.0, 1.0)
arr_pos = np.zeros((T, 2))
arr_pos[:,0] = np.cos(theta)
arr_pos[:,1] = np.sin(theta)
accel_max = math.sqrt((1-0)**2 + (1-0)**2)

T_trans = 8

while(True):
    a = np.random.rand(1)[0]
    b = np.random.rand(1)[0]
    c = np.random.rand(3)
    
    ell_interp = np.linspace((a_old, b_old), (a, b), T_trans)
    col_interp = np.linspace(c_old, c, T_trans, axis=0)
    accel = math.sqrt((a_old-a)**2 + (b_old-b)**2)
    accel_norm = accel/accel_max

    
    for ell_inter, c_inter in zip(ell_interp, col_interp):
        arr_pos_inter = ell_inter*arr_pos
        c_arr = np.repeat(c_inter[np.newaxis, :], T, axis=0)
        #arr_pos_beam = beamify(arr_pos_inter, dwell=4)
        #queue.submit(arr_pos_beam, c_arr)
        queue.submit(arr_pos_inter, c_arr)
        
    a_old = a
    b_old = b
    c_old = c

    

KeyboardInterrupt: 

In [50]:
# Beamify
x = np.linspace(0, 1, 100)
x = np.repeat(x[::10], 10)

In [ ]:
def beamify(arr_pos, num_beams=4, dwell=20):
    '''
    NOT DONE - inserts spots in the trajectory where the beam stops and makes a solid beam
    '''
    T = arr_pos.shape[0]
    beam_idxs = np.linspace(0, T, num_points+2)[1:-1].astype(np.int)
    np.insert(arr_pos)
    return arr_pos

In [78]:
x = np.zeros(101)
num_points = 4
beam_idxs = np.linspace(0, x.shape[0], num_points+2)[1:-1].astype(np.int)

In [77]:
beam_idxs

array([20.2, 40.4, 60.6, 80.8])

In [70]:
def signer(f, T, shift=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0, 1, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t - shift) + 1)/2
    
    
    return x, dx
    
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*np.cos(f_x*theta)
    arr_pos[:,1] = scale_y*np.sin(f_y*theta)
    
    # Not sure what to do about color yet, let's make it white
    arr_col = np.ones((T, 3), np.float32)
    
    return arr_pos, arr_col


In [71]:
arr_pos, arr_col = ellipser()
print(arr_pos.shape, arr_col.shape)

(500, 2) (500, 3)


In [ ]:
# Interpolated ellipses
T = 1000
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

min_vals = np.array([0.8, 0.0, 1.0, 0.0, 5.0])
max_vals = np.array([1.0, 0.4, 2.0, 4.0, 100])
pat_point_old = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals


T_trans = 30

while(True):
    pat_point = max_vals*np.random.rand(len(max_vals))
    ell_interp = np.linspace(pat_point_old, pat_point, T_trans)
    for (scale_x, scale_y, freq_x, freq_y, f_s) in ell_interp:
        arr_pos, arr_col = ellipser(scale_x, scale_y, freq_x, freq_y, f_s)
        queue.submit(arr_pos, arr_col)
        
    pat_point_old = pat_point

In [66]:
T = 1000
arr_col_og = np.ones((T, 3))
x, dx = signer(f=10, T=T, shift=0.75)
theta = 2*np.pi*x
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)


while(True):
    for thresh in np.linspace(0, 1, 100):
        arr_col = arr_col_og*(thresh < (1-dx[:,np.newaxis]))
        queue.submit(arr_pos, arr_col)


KeyboardInterrupt: 

array([False, False, False, False, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False,  True,
        True, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
        True,  True,  True, False, False, False, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
       False, False,  True,  True,  True, False, False, False, False,
       False, False, False,  True,  True,  True, False, False, False,
       False, False, False, False,  True,  True,  True, False, False,
       False, False, False, False, False,  True,  True,  True, False,
       False, False, False, False, False, False,  True,  True,  True,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False, False, False, False, False, False,  True,
        True,  True, False, False, False, False, False, False, False,
        True,  True,